# Прогнозирование стоимости автомобиля по его характеристикам

### Задачи:  
1. Спарсить данные по автомобилям с сайта auto.ru  
2. Подобрать и обучить модель по собранным данным для прогнозирования стоимости авто. Метрика MAPE.



# 1. Tools

In [ ]:
import seaborn as sns
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold, train_test_split
from catboost import CatBoostRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.base import clone
import datetime as dt
from sklearn.feature_selection import f_classif  # проверить значимость переменных
from itertools import combinations
from scipy.stats import ttest_ind  # тест Стьюдента

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

RANDOM_SEED = 42

# 2. Data preparation

In [ ]:
DATA_DIR = '../input/sf-dst-car-price-prediction/'
df_train = pd.read_csv(
    '../input/myparsingfile/parsing_all_moscow_auto_ru_25_11_20.csv')  # спарсил сам
df_train_dop = pd.read_csv(
    '../input/parsing-all-moscow-auto-ru-09-09-2020/all_auto_ru_09_09_2020.csv')  # дополнительный внешний
df_test = pd.read_csv(DATA_DIR+'test.csv')  # тестовый для прогнозирования цен
sample_submission = pd.read_csv(DATA_DIR+'sample_submission.csv')

In [ ]:
df_test.info()

Всего 34686 записей и 32 признака

In [ ]:
df_test.head(3)

In [ ]:
df_train.info()

37 признаков

Всего 66584 записей и 37 признаков

In [ ]:
df_train.head(3)

In [ ]:
df_train_dop.info()

Всего 89378 записей и 26 признаков

In [ ]:
df_train_dop.head(3)

Необходимо:
1. привести признаки color,vehicleTransmission,Руль к одним значениям  
2. склеить df_train и df_train_dop, а также привести количество признаков полученного датасета data к количеству признаков df_test. 
3. привести строковые признаки к нижнему регистру 
4. удалить строки,где bodyType и price is None
5. отобрать строки в data по значениям brand в df_test (уменьшить итоговый датасет train и увеличить точность модели)

In [ ]:
df_train_temp = df_train[df_test.columns]
df_train_temp['price'] = df_train['price']
df_train = df_train_temp

df_train_dop['model_name'] = df_train_dop['model']
df_train_dop['complectation_dict'] = df_train_dop['Комплектация']
df_train_dop.drop(['Комплектация', 'start_date', 'hidden',
                   'model'], axis=1, inplace=True)

df_train_dop['color'] = df_train_dop['color'].map({'CACECB': 'серебристый', 'FAFBFB': 'белый', 'EE1D19': 'красный', '97948F': 'серый',
                                                   '660099': 'пурпурный', '040001': 'чёрный', '4A2197': 'фиолетовый',
                                                   '200204': 'коричневый', '0000CC': 'синий', '007F00': 'зелёный', 'C49648': 'бежевый',
                                                   '22A0F8': 'голубой', 'DEA522': 'золотистый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый',
                                                   'FFC0CB': 'розовый'})

df_train_dop['vehicleTransmission'] = df_train_dop['vehicleTransmission'].map(
    {'AUTOMATIC': 'автоматическая', 'MECHANICAL': 'механическая', 'VARIATOR': 'вариатор', 'ROBOT': 'роботизированная'})
df_train_dop['Руль'] = df_train_dop['Руль'].map(
    {'LEFT': 'левый', 'RIGHT': 'правый'})

for col in df_train.columns:
    if col not in df_train_dop.columns:
        df_train_dop[col] = None

df_train = df_train.append(df_train_dop, sort=False).reset_index(drop=True)

# для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['price'] = 0

data = df_test.append(df_train, sort=False).reset_index(
    drop=True)  # объединяем

data = data[~data.price.isnull()]  # убираем строки None для price
data = data[~data.bodyType.isnull()]  # убираем лишние строки

# привести все не числовые колонки к нижнем регистру(
for col in data.select_dtypes(include='object').columns:
    data[col][~data[col].isnull()] = data[col][~data[col].isnull()
                                               ].apply(lambda x: str(x).lower())

# фильтр по брендам
brands = list(map(lambda x: str.lower(x), df_test.brand.value_counts().index))
data = data[data['brand'].isin(brands)]

# 3. Data cleaning, feature analysis and engineering

In [ ]:
data.info()

In [ ]:
data.select_dtypes(include=['float64']).columns

In [ ]:
for col in data:
    if len(data[data[col].isnull()]) > 0:
        print(col, end=',')

34 колонки , 122370 записей , пропуски в колонках  car_url,complectation_dict,description,equipment_dict,image,model_info,parsing_unixtime,priceCurrency,sell_id,super_gen,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня

Числовые признаки:  'mileage', 'modelDate', 'numberOfDoors', 'productionDate', 'price'.

### Описание признаков:

bodyType - тип кузова       
brand - бренд      
car_url - ссылка на авто     
color - цвет авто   
complectation_dict - описание комплектации    
description - описание от продавца       
engineDisplacement - объем двигателя в литрах  
enginePower - мощность двигателя  
equipment_dict - описание оборудования авто  
fuelType - тип топлива   
image - ссылка на фото авто   
mileage - пробег автомобиля    
modelDate - дата выпуска модели  
model_info - информация об авто  
model_name - модель авто  
name - объем и мощность двигателя    
numberOfDoors - кол-во дверей    
parsing_unixtime - дата парсинга данных     
priceCurrency - валюта продажи  
productionDate - дата производства авто  
sell_id - id объвления о продаже    
super_gen - доп описание авто (клиренс,расход топлива, ценовой сегмент)  
vehicleConfiguration - комбинация типа трансмиссии, объема двигателя и мощности двигателя    
vehicleTransmission - тип трансмиссии (коробки передач)  
vendor - продавец-производитель авто  
Владельцы - кол-во владельцев  
Владение - срок владения автомобилем в годах и месяцах     
ПТС - тип ПТС    
Привод - привод авто     
Руль - руль авто    
Состояние - необходимость проведения ремонта  
Таможня - информация о том расстаможен автомобиль или нет   
sample - признак train/test    
price - цена, указанная в объявлении   


Скопируем датасет, чтобы при необходимости можно было посмотреть данные исходного датасета.

Создадим два списка для обработки категориальных признаков: разбивка на dummy и LabelEncoder

In [ ]:
data_temp = data.copy()

col_dum = []
col_en = []

### Функции

In [ ]:
# Показать график по value_counts (если значений не более 20. Иначе выводить обычный value_counts)
def show(col, vc_show=False, data_x=data_temp):
    vc = data_x[col].value_counts(ascending=True)
    if len(vc) <= 20:
        vc.plot(kind='barh', title=col)
    else:
        print(data_x[col].value_counts())

    if vc_show and len(vc) <= 20:
        print(data_x[col].value_counts())

# Показать распределение на графике для числовых признаков


def show_distplot(col, data_x=data_temp):
    plt.figure()
    sns.distplot(data_x[col][data_x[col] > 0], kde=False, rug=False)
    plt.title(col)
    plt.show()

# логарифмировать признак


def log_f(col, data_x=data_temp):
    data_x[col] = np.log(data_x[col] + 1)

# Проверка на выбросы


def show_outliers(column, data_x=data_temp):
    col = data_x[column]
    IQR = col.quantile(0.75) - col.quantile(0.25)
    perc25 = col.quantile(0.25)
    perc75 = col.quantile(0.75)

    quan_low_outliers = len(data_x.query(f'{column} < {perc25 - 1.5*IQR}'))
    quan_high_outliers = len(data_x.query(f'{column} > {perc75 + 1.5*IQR}'))

    proc_low_outliers = round(quan_low_outliers/len(col) * 100, 2)
    proc_high_outliers = round(quan_high_outliers/len(col) * 100, 2)

    print('Колонка', column)
    print(
        f'Количество выбросов ниже нижней границы: {quan_low_outliers} ({proc_low_outliers}%)')
    print(
        f'Количество выбросов выше верхней границы: {quan_high_outliers} ({proc_high_outliers}%)')
    print('\n')

# привести строку к формату ast или json


def to_dict(row):
    try:
        row = ast.literal_eval(row)
        return row
    except:
        try:
            row = json.loads(row)
            return row
        except:
            return row

# получить значение (ast или json)


def get_val(row, val_name, val_er):
    try:
        return row[val_name]
    except:
        return val_er

### body_type

In [ ]:
show("bodyType")

Больше всего продается внедорожников и седанов.

Признак состоит из названия типа кузова и количества дверей. Количество дверей есть в другом признаке. Оставим только первую часть для группировки значений

In [ ]:
data_temp['bodyType'] = data_temp['bodyType'].apply(
    lambda x: str(x).split(' ')[0])

In [ ]:
show("bodyType")

Склеим похожие значения.

Добавим признак в dummy ,т.к. значений не так много.

In [ ]:
data_temp['bodyType'] = data_temp['bodyType'].apply(
    lambda x: 'купе' if x == 'купе-хардтоп' else x)
data_temp['bodyType'] = data_temp['bodyType'].apply(
    lambda x: 'седан' if x == 'седан-хардтоп' else x)

col_dum.append('bodyType')

### brand

In [ ]:
show("brand")

На рынке доминируют мерседесы.

Добавим признак в dummy ,т.к. значений не так много.

In [ ]:
col_dum.append('brand')

### car_url

In [ ]:
show("car_url")

Вряд ли ссылки на авто на сайте auto.ru повлияют на обучение модели. Удалим признак.

In [ ]:
data_temp.drop('car_url', axis=1, inplace=True)

### color

In [ ]:
show("color")

Больше всего машин с черным цветом.Розовый не в почёте.

Добавим признак в dummy ,т.к. значений не так много.

In [ ]:
col_dum.append('color')

### complectation_dict

In [ ]:
show("complectation_dict")

Заменим пропуски на минимальный размер описания комплектации

In [ ]:
data_temp['complectation_dict'].fillna("{'id': '0'}", inplace=True)

Попробуем привести описание комплектации к его размеру. Проверим на значении метрики

In [ ]:
data_temp['complectation_dict'] = data_temp['complectation_dict'].apply(
    lambda x: len(x))

### description

In [ ]:
show("description")

Заменим пропуски на пустую строку

In [ ]:
data_temp['description'].fillna('[]', inplace=True)

Выведем новые признаки

In [ ]:
data_temp['description_word'] = data_temp['description'].apply(
    lambda x: [i for i in x.split()])

data_temp['salon'] = data_temp['description_word'].apply(lambda x:
                                                         1 if ('рольф' or 'панавто' or 'дилер' or 'кредит' or 'обмен' or 'скид' or 'trade-in' or 'выгода' or 'работаем для вас' or 'автосалон' or 'ликвидация') in x else 0)
data_temp['torg'] = data_temp['description_word'].apply(lambda x:
                                                        1 if ('торг') in x else 0)
data_temp['leather'] = data_temp['description_word'].apply(lambda x:
                                                           1 if ('темный' and 'салон') in x else 0)
data_temp['carter'] = data_temp['description_word'].apply(lambda x:
                                                          1 if ('защита' and 'картера') in x else 0)
data_temp['ABS'] = data_temp['description_word'].apply(lambda x:
                                                       1 if ('антиблокировочная' and 'система') in x else 0)
data_temp['airbags'] = data_temp['description_word'].apply(lambda x:
                                                           1 if ('подушки' and 'безопасности') in x else 0)
data_temp['immob'] = data_temp['description_word'].apply(lambda x:
                                                         1 if ('иммобилайзер') in x else 0)
data_temp['central_locking'] = data_temp['description_word'].apply(lambda x:
                                                                   1 if ('центральный' and 'замок') in x else 0)
data_temp['on_board_computer'] = data_temp['description_word'].apply(lambda x:
                                                                     1 if ('бортовой' and 'компьютер') in x else 0)
data_temp['cruise_control'] = data_temp['description_word'].apply(lambda x:
                                                                  1 if ('круиз-контроль') in x else 0)
data_temp['climat_control'] = data_temp['description_word'].apply(lambda x:
                                                                  1 if ('климат-контроль') in x else 0)
data_temp['multi_rudder'] = data_temp['description_word'].apply(lambda x:
                                                                1 if ('мультифункциональный' and 'руль') in x else 0)
data_temp['power_steering'] = data_temp['description_word'].apply(lambda x:
                                                                  1 if ('гидроусилитель' or 'гидро' or 'усилитель' and 'руля') in x else 0)
data_temp['light_and_rain_sensors'] = data_temp['description_word'].apply(lambda x:
                                                                          1 if ('датчики' and 'света' and 'дождя') in x else 0)
data_temp['сarbon_body_kits'] = data_temp['description_word'].apply(lambda x:
                                                                    1 if ('карбоновые' and 'обвесы') in x else 0)
data_temp['rear_diffuser_rkp'] = data_temp['description_word'].apply(lambda x:
                                                                     1 if ('задний' and 'диффузор') in x else 0)
data_temp['door_closers'] = data_temp['description_word'].apply(lambda x:
                                                                1 if ('доводчики' and 'дверей') in x else 0)
data_temp['rear_view_camera'] = data_temp['description_word'].apply(lambda x:
                                                                    1 if ('камера' or 'видеокамера' and 'заднего' and 'вида') in x else 0)
data_temp['amg'] = data_temp['description_word'].apply(lambda x:
                                                       1 if ('amg') in x else 0)
data_temp['bi_xenon_headlights'] = data_temp['description_word'].apply(lambda x:
                                                                       1 if ('биксеноновые' and 'фары') in x else 0)
data_temp['alloy_wheels'] = data_temp['description_word'].apply(lambda x:
                                                                1 if ('легкосплавные' or 'колесные' or 'диски') in x else 0)
data_temp['parking_sensors'] = data_temp['description_word'].apply(lambda x:
                                                                   1 if ('парктроник' or 'парктронник') in x else 0)
data_temp['dents'] = data_temp['description_word'].apply(lambda x:
                                                         1 if ('вмятины' or 'вмятина' or 'царапина' or 'царапины' or 'трещина') in x else 0)
data_temp['roof_with_panoramic_view'] = data_temp['description_word'].apply(lambda x:
                                                                            1 if ('панорамная' and 'крыша') in x else 0)

data_temp.drop(['description', 'description_word'], axis=1, inplace=True)

In [ ]:
data_temp.info()

### engineDisplacement

In [ ]:
show("engineDisplacement")

Заменим "ltr" на "". Тогда признак получится преобразовать в числовой

In [ ]:
def change_ltr(row):
    row_temp = str(row)
    row_temp = row_temp.replace(' ltr', '')
    try:
        return float(row_temp)
    except:
        return 0


data_temp['engineDisplacement'] = data_temp['engineDisplacement'].apply(
    change_ltr).astype('float64')

### enginePower

In [ ]:
show("enginePower")

Заменим "n12" на "". Тогда признак получится преобразовать в числовой

In [ ]:
def change_ltr(row):
    row_temp = str(row)
    row_temp = row_temp.replace(' n12', '')
    try:
        return int(row_temp)
    except:
        return 0


data_temp['enginePower'] = data_temp['enginePower'].apply(
    change_ltr).astype('int64')

### equipment_dict

In [ ]:
show("equipment_dict")

Заменим пропуски на минимальное значение. Попробуем привести признак к его длине,посмотрим на метрику

In [ ]:
data_temp['equipment_dict'].fillna("{}", inplace=True)
data_temp['equipment_dict'] = data_temp['equipment_dict'].apply(
    lambda x: len(x))

### fuelType

In [ ]:
show("fuelType")

Почти все авто на бензине.

Добавим признак в dummy ,т.к. значений не так много.

In [ ]:
col_dum.append('fuelType')

### image

In [ ]:
show("image")

Многие ссылки на фото повторяются. Есть карточки автомобилей с шаблонными фото авто. Помечаются как "avatars". Разобьем признак на наличие "avatars"

In [ ]:
data_temp['image'].fillna(
    data_temp['image'].value_counts().index[0], inplace=True)
data_temp['image'] = data_temp['image'].apply(
    lambda x: 1 if 'avatars' in x else 0)

### mileage

In [ ]:
show('mileage')

21278 авто с 0 пробегом. Возможно, это новые авто. Или продавец не указал пробег. Оставим пока как есть

In [ ]:
show_outliers('mileage')

Выбросов не много.(1%)

In [ ]:
show_distplot('mileage')

 Логарифмируем признак. Визуально распределение улучшилось

In [ ]:
log_f('mileage')

В целом признак распределен нормально. .

### modelDate

In [ ]:
show('modelDate')

Больше всего авто с датой выпуска за 2017 год

In [ ]:
show_outliers('modelDate')

Выбросов не много.(2+%)

In [ ]:
show_distplot('modelDate')

In [ ]:
log_f('modelDate')

Логорифмирование признака не приводит к нормальному распределнию. Посмотрим на метрику.

### model_info

In [ ]:
show("model_info")

Заполним пропуски и приведем признак к его длине. Посмотрим на метрику.

In [ ]:
data_temp['model_info'].fillna('{}', inplace=True)
data_temp['model_info'] = data_temp['model_info'].apply(lambda x: len(x))

### model_name

In [ ]:
show("model_name")

Уникальных моделей много,около 700. Добавим их для LabelEncoder ,чтобы не перегружать количеством признаков датасет.

In [ ]:
col_en.append('model_name')

### name

In [ ]:
show("name")

Объем и мощность двигателя уже есть в других признаках.Удалим этот признак за ненадобностью.

In [ ]:
data_temp.drop('name', axis=1, inplace=True)

### numberOfDoors

In [ ]:
show("numberOfDoors", vc_show=True)

Больше всего пятидверных авто. Но есть авто и без деверей.(ретро). 

Оставим признак как есть

### parsing_unixtime

In [ ]:
data_temp['parsing_unixtime'].fillna(0, inplace=True)

In [ ]:
show('parsing_unixtime')

In [ ]:
def ut_in_date(row):
    value = dt.datetime.fromtimestamp(int(row))
    return value.strftime('%Y-%m-%d %H:%M:%S')


data_temp['parsing_unixtime'] = data_temp['parsing_unixtime'].apply(ut_in_date)

In [ ]:
show('parsing_unixtime')

Этот признак удаляем. Его можно было бы использовать для расчёта коэффициента влияния курса доллара на цены авто. (разница курса,когда проходил парсинг и когда прогнозируется цена).
В данном случае, курс доллара на дату парсинга данных и на дату прогнозирования практически не отличается.

In [ ]:
data_temp.drop('parsing_unixtime', axis=1, inplace=True)

### priceCurrency

In [ ]:
show('priceCurrency')

Валюта продажи только одна - "rub".  
Удаляем признак

In [ ]:
data_temp.drop('priceCurrency', axis=1, inplace=True)

### productionDate

In [ ]:
show('productionDate')

In [ ]:
show_distplot('productionDate')

Распределение не нормальное. Попробуем логарифмировать

In [ ]:
show_distplot('productionDate')

Распределение не поменялось.Посмотрим на метрику

In [ ]:
show_outliers('productionDate')

Выбросов мало (2%). Оставим как есть

### sell_id

In [ ]:
show('sell_id')

Заполним пропуски 0 и приведём к числовому типу. Попробуем использовать для обучения модели. 

In [ ]:
data_temp['sell_id'].fillna(0, inplace=True)
data_temp['sell_id'] = data_temp['sell_id'].astype('float64')

### super_gen

In [ ]:
show('super_gen')

Заполним пропуски '{}'.   
Попробуем привести признак к формату ast или json.   
Далее вытащим новые признаки: acceleration(ускорение) ,clearance_min,fuel_rate(расход топлива),clearance_max, gear_type ,price_segment

Приведем признак к его длине.

In [ ]:
data_temp['super_gen'].fillna('{}', inplace=True)
data_temp['super_gen'] = data_temp['super_gen'].apply(to_dict)

data_temp['price_segment'] = data_temp['super_gen'].apply(
    lambda x: get_val(x, 'price_segment', 'absent'))
data_temp['gear_type'] = data_temp['super_gen'].apply(
    lambda x: get_val(x, 'gear_type', 'absent'))
data_temp['acceleration'] = data_temp['super_gen'].apply(
    lambda x: get_val(x, 'acceleration', 0))
data_temp['clearance_min'] = data_temp['super_gen'].apply(
    lambda x: get_val(x, 'clearance_min', 0))
data_temp['clearance_max'] = data_temp['super_gen'].apply(
    lambda x: get_val(x, 'clearance_max', 0))
data_temp['fuel_rate'] = data_temp['super_gen'].apply(
    lambda x: get_val(x, 'fuel_rate', 0))

col_dum.append('price_segment')
col_dum.append('gear_type')

data_temp['super_gen'] = data_temp['super_gen'].apply(lambda x: len(x))

### vehicleConfiguration

In [ ]:
show('vehicleConfiguration')

Вся информация есть в других признаках, можно удалить

In [ ]:
data_temp.drop(['vehicleConfiguration'], axis=1, inplace=True)

### vehicleTransmission

In [ ]:
show('vehicleTransmission')

Преобладает автоматическая коробка. Добавим в dummy

In [ ]:
col_dum.append('vehicleTransmission')

### vendor

In [ ]:
show('vendor')

Заполним пропуски

In [ ]:
vendors = {'ford': 'american', 'hyundai': 'korean', 'nissan': 'japanese', 'peugeot': 'european', 'porsche': 'european', 'renault': 'european', 'skoda': 'european', 'toyota': 'japanese', 'volkswagen': 'european',
           'volvo': 'european', 'great_wall': 'chinese', 'land_rover': 'european', 'mercedes': 'european'}
data_temp['vendor'].fillna(data['brand'].map(vendors), inplace=True)

In [ ]:
show('vendor')

In [ ]:
col_dum.append('vendor')

### Владельцы

In [ ]:
show('Владельцы')

Заполним пропуски 0. Будем считать ,что "3 и более" это 4.

In [ ]:
data_temp['Владельцы'].fillna('0', inplace=True)


def change_own(row):
    row = str(row)
    if row.find('или более') > -1:
        return 4
    res = re.match('\d+', row)
    if res is None:
        return row
    else:
        return res[0]


data_temp['Владельцы'] = data_temp['Владельцы'].apply(
    change_own).astype('int64')

In [ ]:
show('Владельцы')

Больше всего у авто было 3 владельца

### Владение

In [ ]:
show('Владение')

В признаке 86271 пропусков. Более 70%. Возможно стоит удалить признак. Посмотрим на метрике.  
Пока попробуем убрать пропуски и определить владение в месяцах

In [ ]:
data_temp['Владение'].fillna('0', inplace=True)


def transform(row):
    year_len = len(re.findall('[гл]', row))
    mounth_len = len(re.findall('[мес]', row))
    res = re.findall('\d+', row)
    res_len = len(res)

    new_row = 0

    if year_len > 0:
        new_row += int(res[0]) * 12

    if mounth_len > 0:
        if res_len == 1:
            new_row += int(res[0])
        elif res_len == 2:
            new_row += int(res[1])

    if year_len == 0 and mounth_len == 0:
        if res_len == 1:
            new_row += int(res[0])
        elif res_len == 2:
            new_row += int(res[1])

    return new_row


data_temp['Владение'] = data_temp['Владение'].apply(transform).astype('int64')

### ПТС

In [ ]:
show('ПТС')

Заменим пропуски на 'оригинал'. Склеим оригинал-original и дубликат-duplicate.
Добавим признак в dummy

In [ ]:
data_temp['ПТС'].fillna('оригинал', inplace=True)
data_temp['ПТС'] = data_temp['ПТС'].apply(lambda x: 'оригинал' if x in [
                                          'original', 'оригинал'] else 'дубликат')
col_dum.append('ПТС')

### Привод

In [ ]:
show('Привод')

In [ ]:
col_dum.append('Привод')

### Руль

In [ ]:
show('Руль')

Авто с правым рулём очень мало. Возможно,стоит удалить признак.Посмотрим метрику.

In [ ]:
data_temp['Руль'].fillna('левый', inplace=True)
col_dum.append('Руль')

### Состояние

In [ ]:
show('Состояние')

Одно значение "не требует ремонта". Удалим признак

In [ ]:
data_temp.drop('Состояние', axis=1, inplace=True)

### Таможня

In [ ]:
show('Таможня')

Одно значение "растаможен". Удалим признак

In [ ]:
data_temp.drop('Таможня', axis=1, inplace=True)

### price

In [ ]:
show('price')

In [ ]:
show_distplot('price')

Подавляющее большинство машин стоит до 1 млн. Распределение не нормальное.

In [ ]:
show_outliers('price')

Выбросов относительно не много,около 8%.Пока не будем удалять.

Логарифмируем цену

In [ ]:
log_f('price')

In [ ]:
show_distplot('price')

Теперь распределение нормальное.

### Dummy и LableEncoder

In [ ]:
# Dummy
for col in col_dum:
    data_temp = pd.get_dummies(data_temp, columns=[col], prefix=col)

# LabelEncoder
label_encoder = LabelEncoder()
for col in col_en:
    data_temp[col] = label_encoder.fit_transform(data_temp[col])

# 4. General analysis

### Функции

In [ ]:
# Корреляция
def show_cor(data_x):
    plt.rcParams['figure.figsize'] = (30, 10)
    sns.heatmap(round(data_x.corr(), 2), annot=True)
    plt.rcParams['figure.figsize'] = (10, 5)

# Значимость признаков


def show_important(data_x, cols):
    df_temp = data_x[data_x['sample'] == 1]
    imp_num = pd.Series(
        f_classif(df_temp[cols], df_temp['price'])[0], index=cols)
    imp_num.sort_values(inplace=True)
    imp_num.plot(
        kind='barh', title='Значимость переменных для прогнозирования price')

# тест Стьюдента


def get_stat_dif(data_x, column):
    cols = data_x.loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(data_x.loc[data_x.loc[:, column] == comb[0], 'price'],
                     data_x.loc[data_x.loc[:, column] == comb[1], 'price']).pvalue \
                > 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Не найдены статистически значимые различия для колонки', column)
            break

In [ ]:
# Выделим числовые признаки
num_cols = ['mileage', 'modelDate', 'numberOfDoors', 'productionDate', 'price', 'complectation_dict', 'engineDisplacement',
            'enginePower', 'equipment_dict', 'model_info', 'sell_id', 'super_gen', 'acceleration', 'clearance_min', 'clearance_max', 'fuel_rate']

**Посмотрим корреляцию для числовых признаков**

In [ ]:
show_cor(data_temp[num_cols])

Высокая корреляция price с признаком super_gen и его субпризнаками.Скорее всего это из-за того,что в этих признаках много пропусков.Это может запутывать модель.Возможно,придется их удалить.

Достаточно высокая корреляция sell_id с price (-0.49). Возможно,на это также влияют пропуски в дополнительно датасете. Посмотрим на метрику.

model_info и sell_id очень сильно коррелируют (0.99). model_info будем удалять,т.к. у него ниже корреляция с price

**Посмотрим значимость переменных для прогнозирования price**

In [ ]:
num_cols.remove('price')

show_important(data_temp, num_cols)

Высокую значимость имеют признаки: mileage,productionDate,modelDate

**тест Стьюдента**

Проверим, есть ли статистическая разница в распределении оценок по категориальным признакам, с помощью теста Стьюдента. Проверим нулевую гипотезу о том, что распределения price по различным параметрам неразличимы.

In [ ]:
test_st_col = []

data_temp_sample = data_temp[data_temp['sample'] == 1]

for col in data_temp_sample.columns:
    if data_temp[col].nunique() == 2:
        test_st_col.append(col)

In [ ]:
for col in test_st_col:
    get_stat_dif(data_temp, col)

Попробуем удалить статистически незначимые признаки и посмотреть результаты метрики.

# 5. Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в несколько больших функций

In [ ]:
# Функции

def to_dict(row):
    try:
        row = ast.literal_eval(row)
        return row
    except:
        try:
            row = json.loads(row)
            return row
        except:
            return row


def get_val(row, val_name, val_er):
    try:
        return row[val_name]
    except:
        return val_er


def change_ltr(row):
    row_temp = str(row)
    row_temp = row_temp.replace(' ltr', '')
    try:
        return float(row_temp)
    except:
        return 0


def change_n12(row):
    row_temp = str(row)
    row_temp = row_temp.replace(' n12', '')
    try:
        return int(row_temp)
    except:
        return 0


def log_f(col, data_x=data_temp):
    data_x[col] = np.log(data_x[col] + 1)


def change_own(row):
    row = str(row)
    if row.find('или более') > -1:
        return 4
    res = re.match('\d+', row)
    if res is None:
        return row
    else:
        return res[0]


def transform(row):
    year_len = len(re.findall('[гл]', row))
    mounth_len = len(re.findall('[мес]', row))
    res = re.findall('\d+', row)
    res_len = len(res)

    new_row = 0

    if year_len > 0:
        new_row += int(res[0]) * 12

    if mounth_len > 0:
        if res_len == 1:
            new_row += int(res[0])
        elif res_len == 2:
            new_row += int(res[1])

    if year_len == 0 and mounth_len == 0:
        if res_len == 1:
            new_row += int(res[0])
        elif res_len == 2:
            new_row += int(res[1])

    return new_row


def mape(y_true, y_pred):
    # Точность модели по метрике MAPE
    return np.mean(np.abs((y_pred-y_true)/y_true)) * 100

In [ ]:
# Данные
DATA_DIR = '../input/sf-dst-car-price-prediction/'
df_train = pd.read_csv(
    '../input/myparsingfile/parsing_all_moscow_auto_ru_25_11_20.csv')
df_train_dop = pd.read_csv(
    '../input/parsing-all-moscow-auto-ru-09-09-2020/all_auto_ru_09_09_2020.csv')
df_test = pd.read_csv(DATA_DIR+'test.csv')
sample_submission = pd.read_csv(DATA_DIR+'sample_submission.csv')
#################################

df_train_temp = df_train[df_test.columns]
df_train_temp['price'] = df_train['price']
df_train = df_train_temp

df_train_dop['model_name'] = df_train_dop['model']
df_train_dop['complectation_dict'] = df_train_dop['Комплектация']
df_train_dop.drop(['Комплектация', 'start_date', 'hidden',
                   'model'], axis=1, inplace=True)

df_train_dop['color'] = df_train_dop['color'].map({'CACECB': 'серебристый', 'FAFBFB': 'белый', 'EE1D19': 'красный', '97948F': 'серый',
                                                   '660099': 'пурпурный', '040001': 'чёрный', '4A2197': 'фиолетовый',
                                                   '200204': 'коричневый', '0000CC': 'синий', '007F00': 'зелёный', 'C49648': 'бежевый',
                                                   '22A0F8': 'голубой', 'DEA522': 'золотистый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый',
                                                   'FFC0CB': 'розовый'})

df_train_dop['vehicleTransmission'] = df_train_dop['vehicleTransmission'].map(
    {'AUTOMATIC': 'автоматическая', 'MECHANICAL': 'механическая', 'VARIATOR': 'вариатор', 'ROBOT': 'роботизированная'})
df_train_dop['Руль'] = df_train_dop['Руль'].map(
    {'LEFT': 'левый', 'RIGHT': 'правый'})

for col in df_train.columns:
    if col not in df_train_dop.columns:
        df_train_dop[col] = None

df_train = df_train.append(df_train_dop, sort=False).reset_index(drop=True)

df_train['sample'] = 1
df_test['sample'] = 0
df_test['price'] = 0

data = df_test.append(df_train, sort=False).reset_index(drop=True)

data = data[~data.price.isnull()]
data = data[~data.bodyType.isnull()]

for col in data.select_dtypes(include='object').columns:
    data[col][~data[col].isnull()] = data[col][~data[col].isnull()
                                               ].apply(lambda x: str(x).lower())

brands = list(map(lambda x: str.lower(x), df_test.brand.value_counts().index))
data = data[data['brand'].isin(brands)]

In [ ]:
# Подготовка данных для обучения и тестирования
data_temp = data.copy()

col_dum = []
col_en = []

# bodyType
data_temp['bodyType'] = data_temp['bodyType'].apply(
    lambda x: str(x).split(' ')[0])
data_temp['bodyType'] = data_temp['bodyType'].apply(
    lambda x: 'купе' if x == 'купе-хардтоп' else x)
data_temp['bodyType'] = data_temp['bodyType'].apply(
    lambda x: 'седан' if x == 'седан-хардтоп' else x)
col_dum.append('bodyType')

# brand
col_dum.append('brand')

# car_url
data_temp.drop('car_url', axis=1, inplace=True)

# color
col_dum.append('color')

# complectation_dict
data_temp['complectation_dict'].fillna("{'id': '0'}", inplace=True)
data_temp['complectation_dict'] = data_temp['complectation_dict'].apply(
    lambda x: len(x))

# description
data_temp['description'].fillna('[]', inplace=True)
data_temp['description_word'] = data_temp['description'].apply(
    lambda x: [i for i in x.split()])

data_temp['salon'] = data_temp['description_word'].apply(lambda x:
                                                         1 if ('рольф' or 'панавто' or 'дилер' or 'кредит' or 'обмен' or 'скид' or 'trade-in' or 'выгода' or 'работаем для вас' or 'автосалон' or 'ликвидация') in x else 0)
data_temp['torg'] = data_temp['description_word'].apply(lambda x:
                                                        1 if ('торг') in x else 0)
data_temp['leather'] = data_temp['description_word'].apply(lambda x:
                                                           1 if ('темный' and 'салон') in x else 0)
data_temp['carter'] = data_temp['description_word'].apply(lambda x:
                                                          1 if ('защита' and 'картера') in x else 0)
data_temp['ABS'] = data_temp['description_word'].apply(lambda x:
                                                       1 if ('антиблокировочная' and 'система') in x else 0)
data_temp['airbags'] = data_temp['description_word'].apply(lambda x:
                                                           1 if ('подушки' and 'безопасности') in x else 0)
data_temp['immob'] = data_temp['description_word'].apply(lambda x:
                                                         1 if ('иммобилайзер') in x else 0)
data_temp['central_locking'] = data_temp['description_word'].apply(lambda x:
                                                                   1 if ('центральный' and 'замок') in x else 0)
data_temp['on_board_computer'] = data_temp['description_word'].apply(lambda x:
                                                                     1 if ('бортовой' and 'компьютер') in x else 0)
data_temp['cruise_control'] = data_temp['description_word'].apply(lambda x:
                                                                  1 if ('круиз-контроль') in x else 0)
data_temp['climat_control'] = data_temp['description_word'].apply(lambda x:
                                                                  1 if ('климат-контроль') in x else 0)
data_temp['multi_rudder'] = data_temp['description_word'].apply(lambda x:
                                                                1 if ('мультифункциональный' and 'руль') in x else 0)
data_temp['power_steering'] = data_temp['description_word'].apply(lambda x:
                                                                  1 if ('гидроусилитель' or 'гидро' or 'усилитель' and 'руля') in x else 0)
data_temp['light_and_rain_sensors'] = data_temp['description_word'].apply(lambda x:
                                                                          1 if ('датчики' and 'света' and 'дождя') in x else 0)
data_temp['сarbon_body_kits'] = data_temp['description_word'].apply(lambda x:
                                                                    1 if ('карбоновые' and 'обвесы') in x else 0)
data_temp['rear_diffuser_rkp'] = data_temp['description_word'].apply(lambda x:
                                                                     1 if ('задний' and 'диффузор') in x else 0)
data_temp['door_closers'] = data_temp['description_word'].apply(lambda x:
                                                                1 if ('доводчики' and 'дверей') in x else 0)
data_temp['rear_view_camera'] = data_temp['description_word'].apply(lambda x:
                                                                    1 if ('камера' or 'видеокамера' and 'заднего' and 'вида') in x else 0)
data_temp['amg'] = data_temp['description_word'].apply(lambda x:
                                                       1 if ('amg') in x else 0)
data_temp['bi_xenon_headlights'] = data_temp['description_word'].apply(lambda x:
                                                                       1 if ('биксеноновые' and 'фары') in x else 0)
data_temp['alloy_wheels'] = data_temp['description_word'].apply(lambda x:
                                                                1 if ('легкосплавные' or 'колесные' or 'диски') in x else 0)
data_temp['parking_sensors'] = data_temp['description_word'].apply(lambda x:
                                                                   1 if ('парктроник' or 'парктронник') in x else 0)
data_temp['dents'] = data_temp['description_word'].apply(lambda x:
                                                         1 if ('вмятины' or 'вмятина' or 'царапина' or 'царапины' or 'трещина') in x else 0)
data_temp['roof_with_panoramic_view'] = data_temp['description_word'].apply(
    lambda x: 1 if ('панорамная' and 'крыша') in x else 0)
data_temp.drop(['description', 'description_word'], axis=1, inplace=True)

# engineDisplacement
data_temp['engineDisplacement'] = data_temp['engineDisplacement'].apply(
    change_ltr).astype('float64')

# enginePower
data_temp['enginePower'] = data_temp['enginePower'].apply(
    change_n12).astype('int64')

# equipment_dict
data_temp['equipment_dict'].fillna("{}", inplace=True)
data_temp['equipment_dict'] = data_temp['equipment_dict'].apply(
    lambda x: len(x))

# fuelType
col_dum.append('fuelType')

# image
data_temp['image'].fillna(
    data_temp['image'].value_counts().index[0], inplace=True)
data_temp['image'] = data_temp['image'].apply(
    lambda x: 1 if 'avatars' in x else 0)

# mileage
# log_f('mileage')

# modelDate
# log_f('modelDate')

# model_info
data_temp['model_info'].fillna('{}', inplace=True)
data_temp['model_info'] = data_temp['model_info'].apply(lambda x: len(x))

# model_name
col_en.append('model_name')

# name
data_temp.drop('name', axis=1, inplace=True)

# numberOfDoors

# parsing_unixtime
data_temp.drop('parsing_unixtime', axis=1, inplace=True)

# priceCurrency
data_temp.drop('priceCurrency', axis=1, inplace=True)

# productionDate

# sell_id
data_temp['sell_id'].fillna(0, inplace=True)
data_temp['sell_id'] = data_temp['sell_id'].astype('float64')

# super_gen  Отсутствие данных ,корреляция с price ,метрика хуже - закомменитровал
# data_temp['super_gen'].fillna('{}',inplace = True)
# data_temp['super_gen'] = data_temp['super_gen'].apply(to_dict)
# data_temp['price_segment'] = data_temp['super_gen'].apply(lambda x: get_val(x, 'price_segment', 'absent'))
# data_temp['gear_type'] = data_temp['super_gen'].apply(lambda x: get_val(x, 'gear_type', 'absent'))
# data_temp['acceleration'] = data_temp['super_gen'].apply(lambda x: get_val(x,'acceleration',0))
# data_temp['clearance_min'] = data_temp['super_gen'].apply(lambda x: get_val(x,'clearance_min',0))
# data_temp['clearance_max'] = data_temp['super_gen'].apply(lambda x: get_val(x,'clearance_max',0))
# data_temp['fuel_rate'] = data_temp['super_gen'].apply(lambda x: get_val(x,'fuel_rate',0))
# col_dum.append('price_segment')
# col_dum.append('gear_type')
#data_temp['super_gen'] = data_temp['super_gen'].apply(lambda x: len(x))
data_temp.drop('super_gen', axis=1, inplace=True)

# vehicleConfiguration
data_temp.drop(['vehicleConfiguration'], axis=1, inplace=True)

# vehicleTransmission
col_dum.append('vehicleTransmission')

# vendor
vendors = {'ford': 'american', 'hyundai': 'korean', 'nissan': 'japanese', 'peugeot': 'european', 'porsche': 'european', 'renault': 'european', 'skoda': 'european', 'toyota': 'japanese', 'volkswagen': 'european',
           'volvo': 'european', 'great_wall': 'chinese', 'land_rover': 'european', 'mercedes': 'european'}
data_temp['vendor'].fillna(data['brand'].map(vendors), inplace=True)
col_dum.append('vendor')

# Владельцы
data_temp['Владельцы'].fillna('0', inplace=True)
data_temp['Владельцы'] = data_temp['Владельцы'].apply(
    change_own).astype('int64')

# Владение
# data_temp['Владение'].fillna('0',inplace = True)
# data_temp['Владение'] = data_temp['Владение'].apply(transform).astype('int64')
data_temp.drop('Владение', axis=1, inplace=True)

# ПТС
data_temp['ПТС'].fillna('оригинал', inplace=True)
data_temp['ПТС'] = data_temp['ПТС'].apply(lambda x: 'оригинал' if x in [
                                          'original', 'оригинал'] else 'дубликат')
col_dum.append('ПТС')

# Привод
data_temp['Привод'].fillna('передний', inplace=True)
col_dum.append('Привод')

# Руль
data_temp['Руль'].fillna('левый', inplace=True)
col_dum.append('Руль')

# Состояние
data_temp.drop('Состояние', axis=1, inplace=True)

# Таможня
data_temp.drop('Таможня', axis=1, inplace=True)

# sample

# price
data_temp['price'] = np.log(data_temp['price'] + 1)

# dummy
for col in col_dum:
    data_temp = pd.get_dummies(data_temp, columns=[col], prefix=col)

# LabelEncoder
label_encoder = LabelEncoder()
for col in col_en:
    data_temp[col] = label_encoder.fit_transform(data_temp[col])

# Удалить по результатам общего анализа (Ухудшает метрику)
# del_col = ['salon','сarbon_body_kits','bodyType_кабриолет','bodyType_лимузин','bodyType_микровэн','bodyType_минивэн','bodyType_пикап','bodyType_родстер','bodyType_тарга','bodyType_фастбек','brand_volkswagen','brand_volvo',
# 'color_коричневый','color_оранжевый','color_розовый','color_серый','fuelType_газ']

#data_temp.drop(del_col,axis = 1,inplace = True)

# 6. Model

In [ ]:
% % time
# RandomForestRegressor # best MAPE ноут/лидерборд : 10.90895 / 11.96442

train_data = data_temp.query('sample == 1').drop(['sample'], axis=1)

X = train_data.drop(['price'], axis=1)
y = train_data['price'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

model = RandomForestRegressor(
    n_estimators=300, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_test = np.round(np.exp(y_test))
y_pred = np.round(np.exp(y_pred))

print(f'MAPE: {np.round(mape(y_test,y_pred),5)}')

# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (5, 5)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

## Подбор параметров 
Не дал лучших результатов метрики на лидерборде

In [ ]:
# train_data = data_temp.query('sample == 1').drop(['sample'], axis=1)

# X = train_data.drop(['price'], axis=1)
# y = train_data['price'].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

# n_estimators = [x for x in range(100, 550, 50)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(10, 110, num=10)]
# max_depth.append(None)
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'bootstrap': bootstrap}

# rf = RandomForestRegressor(random_state=RANDOM_SEED)
# rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=3, verbose=2,
#                                random_state=RANDOM_SEED, n_jobs=-1)
# rf_random.fit(X_train, y_train)

# print(rf_random.best_params_)

{'bootstrap': True,
'max_depth': 98,
'max_features': 'auto',
'n_estimators': 500}

## **Stacking** 
Результат метрики в ноутбуке выше,но на лидерборде ниже. Оставляю только RandomForestRegressor(n_estimators=300, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# def compute_meta_feature(clf, X_train, X_test, y_train, cv):

#     X_meta_train = np.zeros_like(y_train, dtype=np.float32)
#     for train_fold_index, predict_fold_index in cv.split(X_train):
#         X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
#         y_fold_train = y_train[train_fold_index]

#         folded_clf = clone(clf)
#         folded_clf.fit(X_fold_train, y_fold_train)
#         X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)

#     meta_clf = clone(clf)
#     meta_clf.fit(X_train, y_train)

#     X_meta_test = meta_clf.predict(X_test)

#     return X_meta_train, X_meta_test


# def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):

#     features = [
#         compute_meta_feature(clf, X_train, X_test, y_train, cv)
#         for clf in tqdm(classifiers)
#     ]

#     stacked_features_train = np.vstack([
#         features_train for features_train, features_test in features
#     ]).T

#     stacked_features_test = np.vstack([
#         features_test for features_train, features_test in features
#     ]).T

#     return stacked_features_train, stacked_features_test

# models = [
#     RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=RANDOM_SEED),
#     BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED),
#     CatBoostRegressor(iterations = 5000,learning_rate = 0.1,random_seed = RANDOM_SEED,eval_metric='MAPE',logging_level='Silent'),
#     GradientBoostingRegressor(n_estimators=300, random_state=RANDOM_SEED)
# ]

# train_data = data_temp.query('sample == 1').drop(['sample'], axis=1)

# X = train_data.drop(['price'], axis=1)
# y = train_data['price'].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=RANDOM_SEED)

# cv = KFold(n_splits=3, shuffle=True)

# stacked_features_train, stacked_features_test = generate_metafeatures(models, X_train.values, X_test.values, y_train, cv)

# clf = LinearRegression(n_jobs = -1)
# clf.fit(stacked_features_train, y_train)
# y_pred = clf.predict(stacked_features_test)

# y_test = np.round(np.exp(y_test))
# y_pred = np.round(np.exp(y_pred))

# print(f'MAPE: {np.round(mape(y_test, y_pred), 5)}')

# 7. Submission

In [ ]:
X_sub = data_temp.query('sample == 0').drop(['sample', 'price'], axis=1)

predict_submission = model.predict(X_sub)

predict_submission = np.round(np.exp(predict_submission))

sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission.csv', index=False)
sample_submission.head(10)